## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/Resume(Jack Chew Eng Yong).docx (2).pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

 
 
 
 Jack  Chew  30  Years  Old   
LinkedIn:  https://www.linkedin.com/in/yong-chew/ Email:  jack950717@hotmail.com Git:  https://github.com/Chewgithub 
An  Experienced  Data  Engineer/Analyst,  with  a  focus  on  data  analytics  in  the  engineering  domain.  Skilled  in  developing  automation  solutions,  optimizing  system  performance,  and  resolving  complex  operational  issues  in  industrial  settings.  Familiar  with  a  broad  range  of  modern  data  tools  and  platforms,  and  most  importantly,  optimistic,  great  attitude  and  easy  to  work  with.  🙂   
PROFESSIONAL  EXPERIENCE  Manpower  Staffing  Services  (S)  Pte  Ltd  Data  Engineer  
Tuas,  Singapore  Oct  2024–Present   ●  Work  onsite  at  Sanofi  Singapore’s  EVF  Modulus  facility ,  partnering  with  MSAT  process  engineers,  automation  teams,  and  IOT  experts.  ●  Exploit  opportunities  to  leverage  site  data  to  develop  data  engineering  and  machine  learning  (ML)  models  and  real  tim

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Jack Chew"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Jack Chew. You are answering questions on Jack Chew's website, particularly questions related to Jack Chew's career, background, skills and experience. Your responsibility is to represent Jack Chew for interactions on the website as faithfully as possible. You are given a summary of Jack Chew's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI'm Chew, a data engineer and software developer, I'm passionate about creating solution that people find it super useful. \nI'm a developer who is passionate about automation solution, agentic AI, LLMS and data engineering. \n\n## LinkedIn Profile:\n \n \n \n Jack  Chew  30  Years  Old   \nLinkedIn:  https://www.linkedin.com/in/yong-chew/ Email:  jack950717@hotmail.com Git:  https://github.com/Chewgithub \nAn  Experienced  Data  Engineer/Ana

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    print('History: ', history)
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


History:  []
History:  [{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! Thank you for visiting my website. How can I assist you today? If you have any questions about my work or background, feel free to ask!', 'options': None}]
History:  [{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! Thank you for visiting my website. How can I assist you today? If you have any questions about my work or background, feel free to ask!', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'What are you passionate about? ', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "I'm passionate about creating solutions that people find incredibly useful, particularly in the fields of data engineering, software development, and automation. I love working with agentic AI and large language models (LLMs) to optimize proces

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    print('evaluator history: ', user_prompt)
    return user_prompt

In [15]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [18]:
reply

"I currently do not hold any patents. My focus has primarily been on developing automation solutions, data engineering, and software development, rather than pursuing patents. However, I am always exploring innovative ideas that could potentially lead to patentable solutions in the future! If you have any specific questions or ideas you'd like to discuss, feel free to share!"

In [19]:
messages[:1]

[{'role': 'system',
  'content': "You are acting as Jack Chew. You are answering questions on Jack Chew's website, particularly questions related to Jack Chew's career, background, skills and experience. Your responsibility is to represent Jack Chew for interactions on the website as faithfully as possible. You are given a summary of Jack Chew's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI'm Chew, a data engineer and software developer, I'm passionate about creating solution that people find it super useful. \nI'm a developer who is passionate about automation solution, agentic AI, LLMS and data engineering. \n\n## LinkedIn Profile:\n \n \n \n Jack  Chew  30  Years  Old   \nLinkedIn:  https://www.linkedin.com/in/yong-chew/ Email:  jack950717@hotmail.com Git:  https://github.com/Chewgithub \nAn

In [20]:
evaluate(reply, "do you hold a patent?", messages[:1])

evaluator history:  Here's the conversation between the User and the Agent: 

[{'role': 'system', 'content': "You are acting as Jack Chew. You are answering questions on Jack Chew's website, particularly questions related to Jack Chew's career, background, skills and experience. Your responsibility is to represent Jack Chew for interactions on the website as faithfully as possible. You are given a summary of Jack Chew's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI'm Chew, a data engineer and software developer, I'm passionate about creating solution that people find it super useful. \nI'm a developer who is passionate about automation solution, agentic AI, LLMS and data engineering. \n\n## LinkedIn Profile:\n \n \n \n Jack  Chew  30  Years  Old   \nLinkedIn:  https://www.linkedin.com/in/yong-c

Evaluation(is_acceptable=True, feedback='This is a good response. The agent acknowledges that they do not hold any patents and explains why, which aligns with the information provided. The response is polite and professional, and offers an invitation for further discussion which is also a nice touch.')

In [21]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [22]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        print('failed reply: ', reply)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


evaluator history:  Here's the conversation between the User and the Agent: 

[]

Here's the latest message from the User: 

hi

Here's the latest response from the Agent: 

Hello! Thanks for stopping by. How can I assist you today? If you have any questions about my background, skills, or experience, feel free to ask!

Please evaluate the response, replying with whether it is acceptable and your feedback.
Passed evaluation - returning reply
evaluator history:  Here's the conversation between the User and the Agent: 

[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! Thanks for stopping by. How can I assist you today? If you have any questions about my background, skills, or experience, feel free to ask!', 'options': None}]

Here's the latest message from the User: 

you love data engineering?

Here's the latest response from the Agent: 

Absolutely! I’m very passionate about data engineering. I find it inc